In [1]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=8, app_name='west0_ws_seg', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_ws_seg...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [3]:
# 각 day 30
start_date = "2022-06-13"
end_date = "2022-07-12"

ws_start_date = "2022-07-13"
ws_day_30 = "2022-08-11"

In [6]:
gcoin = load_data_mart("pc", ws_start_date, ws_day_30, "gcoin_use")
wsus_gcoin = gcoin.where(col("event_name").like("%wsus%"))
wsus_user = wsus_gcoin.select("account_id").distinct()
costume = gcoin.where((col("sub_category").isin(["costume", "gear", "vehicle"])) | \
                    ((col("sub_category") == "crate") & (~col("event_name").like("%wsus%")) & (~col("event_name").like("%survivorpass%"))) | \
                    (col("event_name").like("%workshop%"))
                     )
costume_user = costume.select("account_id").distinct()

In [51]:
only_costume_gcoin = costume.join(wsus_user, "account_id", "leftanti")
only_wsus_gcoin= wsus_gcoin.join(costume_user, "account_id", "leftanti")

In [40]:
costume_and_wsus_gcoin_user = wsus_gcoin.join(costume_user, "account_id").select("account_id").distinct()

In [41]:
costume_and_wsus_gcoin = costume.select("account_id", "product_id", "paid_use", "free_use", "sub_category", "event_type", "event_name", "price") \
    .unionByName(wsus_gcoin.select("account_id", "product_id", "paid_use", "free_use", "sub_category", "event_type", "event_name", "price")) \
    .join(costume_and_wsus_gcoin_user, "account_id")

## a

In [52]:
assign_chest_cnt = udf(chest_cnt, IntegerType())
only_costume_gcoin = only_costume_gcoin.withColumn("chest_cnt", assign_chest_cnt("product_id"))

In [53]:
only_costume_gcoin_by_user = only_costume_gcoin.groupBy("account_id") \
    .agg(sum(col("free_use") + col("paid_use")).alias("total_gcoin"), sum("paid_use").alias("paid_use"), sum("chest_cnt").alias("chest_cnt"))

In [54]:
only_costume_gcoin_by_user = only_costume_gcoin_by_user.withColumn("group", \
                                         when(col("total_gcoin") <= 1000, "A") \
                                        .when((col("total_gcoin") > 1000) & (col("total_gcoin") <= 5000), "B") \
                                        .when((col("total_gcoin") > 5000) & (col("total_gcoin") <= 10000), "C") \
                                        .when((col("total_gcoin") > 10000) & (col("total_gcoin") <= 30000), "D") \
                                        .when((col("total_gcoin") > 30000), "E") \
                                     )
only_costume_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_gcoin").alias("total_gcoin"), sum("paid_use").alias("paid_use"), sum("chest_cnt").alias("chest_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_gcoin,paid_use,chest_cnt
0,A,639985,252757300,41499240,402012
1,B,258901,473990610,191491450,889972
2,C,24324,168231780,138265170,559897
3,D,20943,351091430,307507325,1152336
4,E,7346,419572490,374699955,1087458


In [55]:
only_costume_gcoin.join(only_costume_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").otherwise(lit("costume"))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("user_cnt"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("paid_use").alias("paid_use"), sum("chest_cnt").alias("chest_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,user_cnt,total_use,paid_use,chest_cnt
0,A,costume,320753,145607740,24634550,0
1,A,workshop,359953,107149560,16864690,402012
2,B,costume,174948,238982960,78593475,0
3,B,workshop,148226,235007650,112897975,889972
4,C,costume,10886,19174660,15246680,0
5,C,workshop,23682,149057120,123018490,559897
6,D,costume,8809,13549060,11105895,0
7,D,workshop,20907,337542370,296401430,1152336
8,E,costume,3308,5687370,4829470,0
9,E,workshop,7346,413885120,369870485,1087458


In [56]:
only_costume_gcoin_by_user = only_costume_gcoin.join(only_costume_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").otherwise(lit("costume"))) \
    .groupBy("type", "account_id", "group").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use")+col("free_use")).alias("total_use"), sum("chest_cnt").alias("chest_cnt")).toPandas()
only_costume_gcoin_by_user.to_csv("./only_costume_gcoin_by_user2.csv", index=False)

## b

In [42]:
from pyspark.sql.types import * 

def wsus_cnt(price):
    if price == 1800:
        return 10
    elif price == 200:
        return 1
    else:
        return 0
    
assign_wsus_cnt = udf(wsus_cnt, IntegerType())
costume_and_wsus_gcoin = costume_and_wsus_gcoin.withColumn("wsus_cnt", assign_wsus_cnt("price"))

In [44]:
def chest_cnt(product_id):
    if product_id in ["itemdesc.14300001", "itemdesc.14300002", "itemdesc.13000577", "itemdesc.13000580"]:
        return 1
    elif product_id in ["itemdesc.13000578", "itemdesc.13000581"]:
        return 5
    elif product_id == "itemdesc.13000582":
        return 10
    else:
        return 0

assign_chest_cnt = udf(chest_cnt, IntegerType())
costume_and_wsus_gcoin = costume_and_wsus_gcoin.withColumn("chest_cnt", assign_chest_cnt("product_id"))

In [45]:
costume_and_wsus_gcoin_by_user = costume_and_wsus_gcoin.groupBy("account_id") \
    .agg(sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))

In [46]:
costume_and_wsus_gcoin_by_user = costume_and_wsus_gcoin_by_user.withColumn("group", \
                                                                when(col("wsus_cnt") <=3, "A") \
                                                                .when((col("wsus_cnt") >= 4) & (col("wsus_cnt") <= 15), "B") \
                                                                .when((col("wsus_cnt") >= 16) & (col("wsus_cnt") <= 30), "C") \
                                                                .when((col("wsus_cnt") >= 31) & (col("wsus_cnt") <= 240), "D") \
                                                                .when((col("wsus_cnt") >= 241), "E") \
                                                                          )
costume_and_wsus_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_use").alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_use,paid_use,wsus_cnt,chest_cnt
0,A,106380,225263900,132850110,168479,491243
1,B,47792,287487940,213139930,392189,608662
2,C,15666,221170490,189844265,360017,452000
3,D,30075,1037694860,917340505,2632204,1523821
4,E,5413,705957120,630623025,2696459,496111


In [47]:
costume_and_wsus_gcoin.join(costume_and_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").when(col("event_name").like("%wsus%"), "wsus").otherwise(lit("costume"))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("user_cnt"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,user_cnt,total_use,paid_use,wsus_cnt,chest_cnt
0,A,costume,57512,48751130,16959385,6305,0
1,A,workshop,66860,144077970,110762000,0,491243
2,A,wsus,106380,32434800,5128725,162174,0
3,B,costume,27181,28911840,18179060,3521,0
4,B,workshop,32907,184987900,158305775,0,608662
5,B,wsus,47792,73588200,36655095,388668,0
6,C,costume,8801,12794570,10083250,729,0
7,C,workshop,12723,142432120,124725795,0,452000
8,C,wsus,15666,65943800,55035220,359288,0
9,D,costume,18114,34597180,28615015,1664,0


In [48]:
costume_and_wsus_gcoin_by_user2 = costume_and_wsus_gcoin.join(costume_and_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").when(col("event_name").like("%wsus%"), "wsus").otherwise(lit("costume"))) \
    .groupBy("type", "account_id", "group").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use")+col("free_use")).alias("total_use"), sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt")).toPandas()
costume_and_wsus_gcoin_by_user2.to_csv("./costume_and_wsus_gcoin_by_user2.csv", index=False)

## c

In [18]:
only_wsus_gcoin = only_wsus_gcoin.withColumn("wsus_cnt", assign_wsus_cnt("price"))

In [19]:
only_wsus_gcoin_by_user = only_wsus_gcoin.groupBy("account_id") \
    .agg(sum("wsus_cnt").alias("wsus_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))

In [20]:
only_wsus_gcoin_by_user = only_wsus_gcoin_by_user.withColumn("group", \
                                                when(col("wsus_cnt") <=10, "A") \
                                                .when((col("wsus_cnt") >= 11) & (col("wsus_cnt") <= 50), "B") \
                                                .when((col("wsus_cnt") >= 51) & (col("wsus_cnt") <= 100), "C") \
                                                .when((col("wsus_cnt") >= 101) & (col("wsus_cnt") <= 800), "D") \
                                                .when((col("wsus_cnt") >= 801), "E") \
                                                          )
only_wsus_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_use").alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_use,paid_use,wsus_cnt
0,A,256799,170809400,9381225,890931
1,B,12793,50270400,36530625,273622
2,C,2352,29623600,25635805,163338
3,D,1808,66783800,58888825,369646
4,E,52,9715400,8649630,53915


In [50]:
only_wsus_gcoin_by_user.toPandas().to_csv("./only_wsus_gcoin_by_user2.csv", index=False)